In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Concatenate

# === Load synthetic data ===
temporal_df = pd.read_csv('temporal_features.csv')
spatial_df = pd.read_csv('spatial_features.csv')

In [2]:
# === Encode labels ===
label_encoder = LabelEncoder()
temporal_df['Label_Encoded'] = label_encoder.fit_transform(temporal_df['Defect_Label'])

# Reshape temporal data: 500 = 100 timesteps × 5 features
TIME_STEPS = 100
NUM_FEATURES = 5
# Sort time columns to ensure correct order
time_columns = sorted([col for col in temporal_df.columns if col.startswith('t')])[:500]
temporal_features = temporal_df[time_columns].values
temporal_reshaped = temporal_features.reshape(-1, TIME_STEPS, NUM_FEATURES)


In [3]:
# Normalize spatial features
spatial_features_df = spatial_df[['Segment_ID', 'Distance', 'Elevation', 'Track_Age']].copy()
scaler = StandardScaler()
spatial_features_df[['Distance', 'Elevation', 'Track_Age']] = scaler.fit_transform(spatial_features_df[['Distance', 'Elevation', 'Track_Age']])

# Merge spatial features using Segment_ID
merged_spatial = temporal_df[['Segment_ID']].merge(spatial_features_df, on='Segment_ID', how='left')
spatial_features = merged_spatial[['Distance', 'Elevation', 'Track_Age']].values

# Extract labels
labels = temporal_df['Label_Encoded'].values


In [4]:
# Train-test split
X_temp_train, X_temp_test, X_spat_train, X_spat_test, y_train, y_test = train_test_split(
    temporal_reshaped, spatial_features, labels, test_size=0.2, stratify=labels, random_state=42
)

In [5]:
# === Define Dual-Branch Model ===
temporal_input = Input(shape=(TIME_STEPS, NUM_FEATURES), name='Temporal_Input')
x1 = LSTM(64, return_sequences=False)(temporal_input)
x1 = Dropout(0.3)(x1)

spatial_input = Input(shape=(3,), name='Spatial_Input')
x2 = Dense(32, activation='relu')(spatial_input)
x2 = Dropout(0.3)(x2)


In [6]:
combined = Concatenate()([x1, x2])
x = Dense(32, activation='relu')(combined)
output = Dense(len(np.unique(labels)), activation='softmax')(x)

model = Model(inputs=[temporal_input, spatial_input], outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [7]:
# Train
model.fit(
    [X_temp_train, X_spat_train], y_train,
    validation_data=([X_temp_test, X_spat_test], y_test),
    epochs=20, batch_size=16, verbose=2
)

# Summary
model.summary()

Epoch 1/20
150/150 - 2s - 14ms/step - accuracy: 0.3329 - loss: 1.3146 - val_accuracy: 0.4167 - val_loss: 1.2303
Epoch 2/20
150/150 - 1s - 9ms/step - accuracy: 0.4358 - loss: 1.1822 - val_accuracy: 0.5500 - val_loss: 1.1085
Epoch 3/20
150/150 - 1s - 8ms/step - accuracy: 0.4896 - loss: 1.0934 - val_accuracy: 0.5267 - val_loss: 1.0171
Epoch 4/20
150/150 - 1s - 8ms/step - accuracy: 0.5412 - loss: 1.0157 - val_accuracy: 0.6250 - val_loss: 0.9396
Epoch 5/20
150/150 - 1s - 8ms/step - accuracy: 0.5529 - loss: 0.9722 - val_accuracy: 0.6483 - val_loss: 0.8886
Epoch 6/20
150/150 - 9s - 57ms/step - accuracy: 0.5883 - loss: 0.9253 - val_accuracy: 0.6500 - val_loss: 0.8408
Epoch 7/20
150/150 - 3s - 17ms/step - accuracy: 0.6012 - loss: 0.8947 - val_accuracy: 0.7150 - val_loss: 0.8024
Epoch 8/20
150/150 - 1s - 8ms/step - accuracy: 0.6292 - loss: 0.8606 - val_accuracy: 0.6983 - val_loss: 0.7739
Epoch 9/20
150/150 - 1s - 8ms/step - accuracy: 0.6400 - loss: 0.8210 - val_accuracy: 0.7050 - val_loss: 0.747

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Temporal_Input      │ (None, 100, 5)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Spatial_Input       │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 64)        │     17,920 │ Temporal_Input[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 32)        │        128 │ Spatial_Input[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 64)        │          0 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 32)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 96)        │          0 │ dropout[0][0],    │
│ (Concatenate)       │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │      3,104 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 4)         │        132 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 63,854 (249.43 KB)

 Trainable params: 21,284 (83.14 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 42,570 (166.29 KB)

In [10]:
from sklearn.metrics import accuracy_score, classification_report

# Predict
y_pred_probs = model.predict([X_temp_test, X_spat_test])
y_pred = np.argmax(y_pred_probs, axis=1)

# Accuracy
print("Test Accuracy:", accuracy_score(y_test, y_pred))

# Classification Report
print("\nClassification Report:")
target_names = [str(cls) for cls in label_encoder.classes_]
print(classification_report(y_test, y_pred, target_names=target_names))


19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step
Test Accuracy: 0.7533333333333333

Classification Report:
              precision    recall  f1-score   support

   Corrosion       0.75      0.94      0.83       108
       Crack       0.69      0.68      0.68       192
Misalignment       0.86      0.76      0.81       168
         nan       0.73      0.70      0.72       132

    accuracy                           0.75       600
   macro avg       0.76      0.77      0.76       600
weighted avg       0.76      0.75      0.75       600

